In [ ]:
from z3 import *
from itertools import combinations
import time
import numpy as np

# Aux functions

def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin

def at_least_one(bool_vars):
    return Or(bool_vars)

# def at_most_one(bool_vars, name):
#     constraints = []
#     n = len(bool_vars)
#     m = math.ceil(math.log2(n))
#     r = [Bool(f"r_{name}_{i}") for i in range(m)]
#     binaries = [toBinary(i, m) for i in range(n)]
#     for i in range(n):
#         for j in range(m):
#             phi = Not(r[j])
#             if binaries[i][j] == "1":
#                 phi = r[j]
#             constraints.append(Or(Not(bool_vars[i]), phi))        
#     return And(constraints)


def at_most_one(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one(bool_vars, name):
    return And(at_most_one(bool_vars, name),at_least_one(bool_vars))

def exactly_zero(bool_vars):
    return Not(Or(bool_vars))

# Comparing two boolean arrays - one is less_eq to another if the True value appears first
# lesseq([x1,x2,x3], [y1,y2,y3]) = (x1 or !y1) and (x1==y1 => x2 or ! y2) and (...)
def lesseq(x, y): 
    return And(
            [Or(x[0],Not(y[0]))] +
            [Implies(
                And([x[j] == y[j] for j in range(i)]),
                Or(x[i],Not(y[i]))
            ) for i in range(1, len(x))])

# lesseq for n blocks
# n_lesseq([x1,x2,x3], [y1,y2,y3],2) = lesseq(x1,y1) and (x1_block1==y1_block1 => lesseq(x2,y2)) and ...
def n_lesseq(x, y, n): 
    return And([lesseq(x[0], y[0])] +
               [Implies(
                    And([And([x[j][k] == y[j][k] for k in range(n)]) for j in range(i)]),
                    lesseq(x[i], y[i])
                ) for i in range(1, len(x))])



# Instance 
class Instance(object):
    width = 0
    n = 0
    dimensions = []
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions


# Read instances: 
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            line = f.readline()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance


def solve(instance):
    solver = Solver()

    # Variable Initalization

    x_dims=[]
    y_dims=[]

    for x_dim,y_dim in instance.dimensions:
        x_dims.append(x_dim)
        y_dims.append(y_dim )

    max_height = math.ceil(sum(y_dims)/(instance.width//max(x_dims)))
    min_height = math.ceil(sum([x_dims[i] * y_dims[i] for i in range(instance.n)]) / instance.width)

    print("Width of the plate: ")
    print(instance.width)
    print("Max height of the plate: ")
    print(max_height)
    print("Min height of the plate: ")
    print(min_height) 
    # Solve
    
    solver.set('timeout', 500 * 1000)

    satisfiable = False
    
    
    for height in range(min_height, max_height):
        solver.push()
                    # Plate width * max_height. Each position has n (nr of blocks) boolean values
        boolean_plate = [[[Bool("coord_b"+str(b+1)+"_x"+str(x)+"_y"+str(y)) for b in range(instance.n)] for x in range(instance.width) ] for y in range(height)]  

        # Constraints

        # Blocks must not overlap together

        for y in range(height):
            for x in range(instance.width):
                solver.add(at_most_one(boolean_plate[y][x], f"unique_{y}_{x}"))     # Only one True on each plate cell


        # One hot encoding of height 

#         solution_height = [Bool("h"+str(height)) for height in range(min_height - 1, height)]   # [h1,h2,h3,h4,...]

    #     solver.add(exactly_one(solution_height, "one"))    # Only one value True on solution_height

#         for y in range(min_height - 1, height):
#             solver.add(solution_height[y - (min_height - 1)] == And(at_least_one(list(np.ravel(boolean_plate[y])))
#                                                  , exactly_zero(list(np.ravel(boolean_plate[y+1:])))))


        for b in range(instance.n):
            formula = []
            for y in range(height - y_dims[b]+1):
                for x in range(instance.width - x_dims[b]+1):
                    formula.append(And([boolean_plate[j][i][b] for j in range(y, y+y_dims[b]) for i in range(x,x+x_dims[b])]))
            solver.add(exactly_one(formula, f"formula_{b}"))       


        # Symmetry Breaking 

        # Flips
        '''
        
        # Horizontal flip
        solver.add( [n_lesseq(
                        [boolean_plate[i][j] for j in range(instance.width) for i in range(height)],
                        [boolean_plate[i][j] for j in reversed(range(instance.width)) for i in range(height)], 
                        instance.n)
                     ] )

        # Vertical flip
        solver.add( [n_lesseq(
                        [boolean_plate[i][j] for j in range(instance.width) for i in range(height)],
                        [boolean_plate[i][j] for j in range(instance.width) for i in reversed(range(height))], 
                        instance.n)
                     ] )

        # Horizontal + Vertical flip
        solver.add( [n_lesseq(
                        [boolean_plate[i][j] for j in range(instance.width) for i in range(height)],
                        [boolean_plate[i][j] for j in reversed(range(instance.width)) for i in reversed(range(height))], 
                        instance.n)
                     ] )
        '''

        # Placing the biggest area block on the corner 0,0

        areas = []
        for block in range(instance.n):
            areas.append(x_dims[block]*y_dims[block])

        block_biggest_area = np.argmax(np.asarray(areas))

        solver.add(boolean_plate[0][0][block_biggest_area]==True)

#         solver.add(solution_height[i - (min_height-1)])
        print("starting _solving")
        if solver.check() == sat:
            model = solver.model()
            satisfiable = True
            break
        solver.pop()
    if satisfiable: 
        print("Satisfiable")
        print("Solution - plate height:")
        print(height)
    else: print("Not satisfiable")

    return solver, boolean_plate

def main():
    instance_file = "..\instances\ins-30.txt"
    instance = read_file(instance_file)

    start = time.time()
    solver,boolean_plate = solve(instance)
    end = time.time()

    print("{:.2f}".format(end - start) + " seconds")


if __name__ == '__main__':
    main()

Width of the plate: 
37
Max height of the plate: 
99
Min height of the plate: 
37
starting _solving
starting _solving


In [ ]:
solver = Solver()


# Comparing two boolean arrays - one is less_eq to another if the True value appears first
# lesseq([x1,x2,x3], [y1,y2,y3]) = (x1 or !y1) and (x1==y1 => x2 or ! y2) and (...)
def lesseq(x, y): 
    return And(
            [Or(x[0],Not(y[0]))] +
            [Implies(
                And([x[j] == y[j] for j in range(i)]),
                Or(x[i],Not(y[i]))
            ) for i in range(1, len(x))])

# lesseq for n blocks
# n_lesseq([x1,x2,x3], [y1,y2,y3],2) = lesseq(x1,y1) and (x1_block1==y1_block1 => lesseq(x2,y2)) and ...
def n_lesseq(x, y, n): 
    return And([lesseq(x[0], y[0])] +
               [Implies(
                    And([And([x[j][k] == y[j][k] for k in range(n)]) for j in range(i)]),
                    lesseq(x[i], y[i])
                ) for i in range(1, len(x))])

solver.add(n_lesseq([[True,False],[True,False]],[[False,True],[True,False]],2))
if solver.check() == sat: 
    print(solver.model)
    print("ok") 
else: 
    print("not less")

In [ ]:
x = [1,2,3]
x[1:3]